In [ ]:
#Code taken from https://github.com/iwantooxxoox/Keras-OpenFace

In [1]:
from model import create_model

nn4_small2 = create_model()

Using TensorFlow backend.


In [2]:
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Layer

in_a = Input(shape=(96, 96, 3))
in_p = Input(shape=(96, 96, 3))
in_n = Input(shape=(96, 96, 3))

emb_a = nn4_small2(in_a)
emb_p = nn4_small2(in_p)
emb_n = nn4_small2(in_n)

print(emb_a)
class TripletLossLayer(Layer):
    def __init__(self, alpha, **kwargs):
        self.alpha = alpha
        super(TripletLossLayer, self).__init__(**kwargs)
    
    def triplet_loss(self, inputs):
        a, p, n = inputs
        p_dist = K.sum(K.square(a-p), axis=-1)
        n_dist = K.sum(K.square(a-n), axis=-1)
        return K.sum(K.maximum(p_dist - n_dist + self.alpha, 0), axis=0)
    
    def call(self, inputs):
        loss = self.triplet_loss(inputs)
        self.add_loss(loss)
        return loss


# Layer that computes the triplet loss from anchor, positive and negative embedding vectors
triplet_loss_layer = TripletLossLayer(alpha=0.2, name='triplet_loss_layer')([emb_a, emb_p, emb_n])


# Model that can be trained with anchor, positive negative images
nn4_small2_train = Model([in_a, in_p, in_n], triplet_loss_layer)

print(nn4_small2_train)

Tensor("model_1/norm_layer/l2_normalize:0", shape=(?, 128), dtype=float32)


In [3]:
from data import triplet_generator

generator = triplet_generator() 

nn4_small2_train.compile(loss=None, optimizer='adam')

nn4_small2_train.fit_generator(generator, epochs=1, steps_per_epoch=100)


Epoch 1/1
100/100 [==============================] - 171s 2s/step - loss: 0.7929


In [4]:
nn4_small2_train.save_weights('my_model_weights.h5')
x = create_model()